In [170]:
# Import Pandas library
import pandas as pd

# Load Data
school_data_file = "Resources/schools_complete.csv"
student_data_file = "Resources/students_complete.csv"

# Read School and Student Data into Data Frames
school_data = pd.read_csv(school_data_file)
student_data = pd.read_csv(student_data_file)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name","school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [171]:
#Calculate the total number of schools
total_schools = len(school_data_complete["school_name"].unique())

#Calculate the total number of students
total_students = school_data_complete["student_name"].count()

#Calculate the total budget
total_budget = school_data_complete["budget"].sum()

#Calculate the average math score
avg_math_score = school_data_complete["math_score"].mean()

#Calculate the average reading score
avg_reading_score = school_data_complete["reading_score"].mean()

#Calculate the percentage of students with a passing math score (70 or greater)
math_passing_percent = (school_data_complete.loc[school_data_complete["math_score"] >= 70]["student_name"].count())/total_students

#Calculate the percentage of students with a passing reading score (70 or greater)
reading_passing_percent = (school_data_complete.loc[school_data_complete["reading_score"] >= 70]["student_name"].count())/total_students

#Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
overall_passing_rate = (avg_math_score + avg_reading_score)/2

#Create a dataframe to hold the above results
district_summary = pd.DataFrame({"Total Schools":[total_schools],
                                "Total Students": [total_students],
                                "Total Budget":[total_budget],
                                "Average Math Score": [avg_math_score],
                                "Average Reading Score": [avg_reading_score],
                                "Math Passing %": [math_passing_percent * 100],
                                "Reading Passing %": [reading_passing_percent * 100],
                                "Overall Passing Rate":[overall_passing_rate]})

#Optional: give the displayed data cleaner formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].astype(float).map("${:,.2f}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:,.3f}".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:,.3f}".format)
district_summary["Math Passing %"] = district_summary["Math Passing %"].map("{:,.2f}%".format)
district_summary["Reading Passing %"] = district_summary["Reading Passing %"].map("{:,.2f}%".format)
district_summary["Overall Passing Rate"] = district_summary["Overall Passing Rate"].map("{:,.3f}".format)
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Math Passing %,Reading Passing %,Overall Passing Rate
0,15,"39,170","$82,932,329,558.00",78.985,81.878,74.98%,85.81%,80.432


## School Summary

In [172]:
##Create an overview table that summarizes key metrics about each school, including:
#School Name

#School Type
school_types = school_data_complete.groupby('school_name')['type'].unique().apply(lambda types: ''.join(map(str,types)))

#Total Students
school_students = school_data_complete.groupby('school_name')['student_name'].count()

#Total School Budget
school_budgets = school_data_complete.groupby('school_name')['budget'].mean()

#Per Student Budget
school_per_student_budget = school_budgets/school_students

#Average Math Score
school_avg_math = school_data_complete.groupby('school_name')['math_score'].mean()

#Average Reading Score
school_avg_reading = school_data_complete.groupby('school_name')['reading_score'].mean()

# Calculate passing rate
def passing_rate(scores):
    pass_count = 0
    students = len(scores)
    for score in scores:
        if score >= 70: pass_count += 1
    return (pass_count/students) * 100

#% Passing Math
school_math_passing = school_data_complete.groupby('school_name')['math_score'].apply(passing_rate)

#% Passing Reading
school_reading_passing = school_data_complete.groupby('school_name')['reading_score'].apply(passing_rate)

#Overall Passing Rate (Average of the above two)
school_overall_passing = (school_math_passing + school_reading_passing)/2

In [173]:
##Create a dataframe to hold the above results
schools_summary = pd.concat([school_types, school_students, school_budgets, school_per_student_budget, school_avg_math, school_avg_reading, school_math_passing, school_reading_passing, school_overall_passing], axis=1)
schools_summary.columns = ['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', 'Average Reading Score', 'Math Passing %', 'Reading Passing %', 'Overall Passing Rate']

schools_summary["Total Students"] = schools_summary["Total Students"].map("{:,}".format)
schools_summary["Total School Budget"] = schools_summary["Total School Budget"].astype(float).map("${:,.2f}".format)
schools_summary["Per Student Budget"] = schools_summary["Per Student Budget"].astype(float).map("${:,.2f}".format)
schools_summary["Average Math Score"] = schools_summary["Average Math Score"].map("{:,.3f}".format)
schools_summary["Average Reading Score"] = schools_summary["Average Reading Score"].map("{:,.3f}".format)
schools_summary["Math Passing %"] = schools_summary["Math Passing %"].map("{:,.2f}%".format)
schools_summary["Reading Passing %"] = schools_summary["Reading Passing %"].map("{:,.2f}%".format)
schools_summary["Overall Passing Rate"] = schools_summary["Overall Passing Rate"].map("{:,.3f}".format)

schools_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Math Passing %,Reading Passing %,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048,81.034,66.68%,81.93%,74.307
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.062,83.976,94.13%,97.04%,95.587
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.712,81.158,65.99%,80.74%,73.364
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.103,80.746,68.31%,79.30%,73.804
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351,83.817,93.39%,97.14%,95.266
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.290,80.934,66.75%,80.86%,73.808
Holden High School,Charter,427,"$248,087.00",$581.00,83.803,83.815,92.51%,96.25%,94.379
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629,81.183,65.68%,81.32%,73.500
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072,80.966,66.06%,81.22%,73.640


## Top Performing Schools (By Passing Rate)

In [174]:
top_performing_schools = schools_summary.sort_values(["Overall Passing Rate"], ascending=False)
top_performing_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Math Passing %,Reading Passing %,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.062,83.976,94.13%,97.04%,95.587
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418,83.849,93.27%,97.31%,95.291
Pena High School,Charter,962,"$585,858.00",$609.00,83.840,84.045,94.59%,95.95%,95.270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351,83.817,93.39%,97.14%,95.266
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274,83.989,93.87%,96.54%,95.204


## Bottom Performing Schools (By Passing Rate)

In [175]:
low_performing_schools = schools_summary.sort_values(["Overall Passing Rate"], ascending=True)
low_performing_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Math Passing %,Reading Passing %,Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.843,80.745,66.37%,80.22%,73.293
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.712,81.158,65.99%,80.74%,73.364
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629,81.183,65.68%,81.32%,73.500
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072,80.966,66.06%,81.22%,73.640
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.103,80.746,68.31%,79.30%,73.804


## Math Scores by Grade

In [176]:
schools_9th_math = school_data_complete[school_data_complete['grade']=='9th'].groupby(['school_name'])['math_score'].mean()
schools_10th_math = school_data_complete[school_data_complete['grade']=='10th'].groupby(['school_name'])['math_score'].mean()
schools_11th_math = school_data_complete[school_data_complete['grade']=='11th'].groupby(['school_name'])['math_score'].mean()
schools_12th_math = school_data_complete[school_data_complete['grade']=='12th'].groupby(['school_name'])['math_score'].mean()
schools_math = pd.concat([schools_9th_math, schools_10th_math, schools_11th_math, schools_12th_math], axis=1)
schools_math.columns = ['9th', '10th', '11th', '12th']
schools_math

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

In [177]:
schools_9th_reading = school_data_complete[school_data_complete['grade']=='9th'].groupby(['school_name'])['reading_score'].mean()
schools_10th_reading = school_data_complete[school_data_complete['grade']=='10th'].groupby(['school_name'])['reading_score'].mean()
schools_11th_reading = school_data_complete[school_data_complete['grade']=='11th'].groupby(['school_name'])['reading_score'].mean()
schools_12th_reading = school_data_complete[school_data_complete['grade']=='12th'].groupby(['school_name'])['reading_score'].mean()
schools_reading = pd.concat([schools_9th_reading, schools_10th_reading, schools_11th_reading, schools_12th_reading], axis=1)
schools_reading.columns = ['9th', '10th', '11th', '12th']
schools_reading

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [178]:
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [179]:
schools_spending = pd.concat([school_per_student_budget, school_avg_math, school_avg_reading, school_math_passing, school_reading_passing, school_overall_passing], axis=1)
schools_spending.columns = ["Per Student Budget", "Average Math Score", "Average Reading Score", "Math Passing %", "Reading Passing %", "Overall Passing Rate"]
schools_spending["Spending Ranges (Per Student)"] = pd.cut(schools_spending['Per Student Budget'],  bins=spending_bins, labels=group_names)
del schools_spending['Per Student Budget']
schools_spending.groupby(["Spending Ranges (Per Student)"]).mean()

,Average Math Score,Average Reading Score,Math Passing %,Reading Passing %,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Type

In [180]:
schools_type = pd.concat([school_types, school_avg_math, school_avg_reading, school_math_passing, school_reading_passing, school_overall_passing], axis=1)
schools_type.columns = ["School Type", "Average Math Score", "Average Reading Score", "Math Passing %", "Reading Passing %", "Overall Passing Rate"]
schools_type.groupby(['School Type']).mean()

,Average Math Score,Average Reading Score,Math Passing %,Reading Passing %,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
